In [1]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import Dense, Dropout, Input
from keras.models import Sequential

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
VIRTUAL_HEIGHT = int(os.environ["VIRTUAL_HEIGHT"])
VIRTUAL_WIDTH = int(os.environ["VIRTUAL_WIDTH"])
DIM = VIRTUAL_WIDTH * VIRTUAL_HEIGHT
BATCH_SIZE = 64
MOVE_CODE = {0:"right", 1:"left", 2:"up", 3:"down"}

## Data stuff

In [3]:
from glob import glob
from objects import Table, Food, Snake
import pickle as pkl
def load_dataset(folder_path=f"dataset_{VIRTUAL_HEIGHT}x{VIRTUAL_WIDTH}", maximum=None):
    paths = glob(folder_path + "/*")

    games = []
    for path in paths[:maximum if maximum else len(paths)]:
        with open(path, "rb") as fin:
            table = pkl.load(fin)
            games.append(table)
            
    return games

init_games = load_dataset(maximum=None)
print(f"Loaded {len(init_games)} games")

Loaded 500 games


## Train feeder

In [5]:
from utils import TrainFeeder
    
feeder = TrainFeeder(init_games=init_games)

## Model

In [10]:
from model import snakemodel
snakemodel.feeder = feeder

In [11]:
snakemodel.compile(optimizer="adam", run_eagerly=True)
snakemodel.summary()

Model: "snake_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                4112      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
Total params: 69,972
Trainable params: 69,972
Non-trainable params: 0
_________________________________________________________________


In [8]:
import logging
logging.basicConfig(format="%(asctime)s || %(message)",
                    level=logging.DEBUG)

In [12]:
from keras.callbacks import ModelCheckpoint

history = snakemodel.fit(x=feeder.feed,
    epochs=3,
    verbose=1,
    callbacks=[ModelCheckpoint("models/epoch_{epoch}.h5", verbose=False)],
    steps_per_epoch=100,
)

Epoch 1/3
100/100 [==============================] - 4s 45ms/step - mean_score: 1.3652
Epoch 2/3
100/100 [==============================] - 4s 44ms/step - mean_score: 1.3234
Epoch 3/3
100/100 [==============================] - 4s 43ms/step - mean_score: 1.2573


In [10]:
with open("models/history.pkl", "wb") as fout:
    pkl.dump(history.history, fout)

FileNotFoundError: [Errno 2] No such file or directory: 'models/history.pkl'